# Income's Effect on Pregnancy: Poisson & Negative Binomial Regression
In this project, I will answer the question: How does household income and other variables influence the number of pregnancies a woman may have?

Note that a woman’s number of pregnancies has a broader definition than number of children; it includes pregnancies that did not carry to term and stillbirths.

We will start by defining our Null Hypothesis: 
There is no significant correlation between income, race, or other variables on the number of pregnancies.


## Data
The sample was taken from the The Study of Women's Health Across the Nation (SWAN), a research endeavor that collected longitudinal data regarding health metrics from a pool of diverse middle aged women, particularly in regards to their menopausal transition. The data  used in this study were taken from their baseline dataset, which collected in the United States from 3,302 participants who joined the study 1996-1997.

[SWAN](https://www.icpsr.umich.edu/web/NACDA/studies/28762#)


|  | Sample Size |
| ----------- | ----------- |
| Initial | 3302 |
| After Dropping Missing | 3266 |

Missing Values for Number of Pregnancies will be dropped, and no other missing values were found bar the following; Missing Total Family Income, Sexually Active, and Contraceptive Use. These missing values made a significant portion of the sample, and as such, Dummy Variables were created for these missing values

### Our Dependant Variable
Number of Pregnancies
NUMPREG0: The number of pregnancies a woman has had as of the date the survey was taken; includes unsuccessful pregnancies in addition to successful births.

### Other Variables
Household Income
The Household Income in the past 12 months of each woman who was surveyed; not the same as personal income. Separated into five dummy variables.

| dummy | Amount $ |
| ----------- | ----------- |
| xincm | Missing Values |
| xinc1 | Less than 19,999 |
| xinc2 | 20,000 to 49,999 |
| xinc3 | 50,000 to 99,000 |
| xinc4 | 100,000 or More |

Race
The Race of each woman who was surveyed, separated into four dummy variables with white omitted. There are no missing variables; only white, black, japanese, chinese, or hispanic women were surveyed.

Age
The Age of each woman at the time of survey. The SWAN study only only targeted middle aged women, therefore values range from 42 to 53 years old. No missing values.

Contraception
Whether or not a woman used any methods to prevent pregnancy during the past six months before the survey was taken. Three dummy variables were created: yes, no, or missing.
| dummy | Taking contraception? |
| ----------- | ----------- |
| conm | Missing Values |
| con0 | No |
| con1 | Yes |

Commited Relationship
COMMITE0: Whether or not the woman was in a committed relationship with a significant other; includes relationships other than marriage. No missing values in data.

Sexually Active
Whether or not the woman was sexually active during the past six months before the survey was taken. Three dummy variables were created; yes, no, or missing.
| dummy | Sexually Active? |
| ----------- | ----------- |
| sexam | Missing Values |
| sexa0 | No |
| sex1 | Yes |


## Cleaning Missing Values

In [207]:
# Load necessary libraries
library(tidyverse)
library(haven)
library(broom)
library(stats)
library(MASS)

# Load the dataset
data <- read_dta("28762-0001-Data.dta")

#Initial sample size
nrow(data)

[1] 3302

The initial sample size is 3302, but it requires cleaning. Missing/invalud values are present, but are notated with either missing cells and/or certain negative values. Certain binary variables must also be changed to match the values of 0 and 1.

Before dropping missing values or creating dummy variables, I will recode all missing value notation to -9999 to clean them.

In [208]:
#NUMPREG0 table
table(data$NUMPREG0)


 -9   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15 
  9 361 340 746 711 520 319 148  74  28  21  12   4   2   5   1   1 

Here we see an example of the missing/invalid value notation used in the original data (-9). I will now proceed to clean the rest of the data.

In [209]:
# Recode NUMPREG0
data <- data %>% mutate(NUMPREG0 = ifelse(NUMPREG0 == -9, -9999, NUMPREG0))

# Recode INCOME0
data <- data %>% mutate(
  INCOME0 = case_when(
    INCOME0 %in% c(-9, -8, -7) ~ -9999,
    is.na(INCOME0) ~ -9999,
    TRUE ~ INCOME0
  )
)

# Generate `insample` column
data <- data %>% mutate(insample = ifelse(NUMPREG0 == -9999, 0, 1))

#INCOME0 table
data %>%
  filter(insample == 1) %>%
  count(INCOME0)

# Summary statistics for NUMPREG0 if insample
num_preg_summary <- data %>% filter(insample == 1) %>% summarise(across(NUMPREG0, list(mean = mean, sd = sd, min = min, max = max), na.rm = TRUE))
num_preg_summary

INCOME0,n
<dbl+lbl>,<int>
-9999,89
1,488
2,1086
3,1160
4,470


NUMPREG0_mean,NUMPREG0_sd,NUMPREG0_min,NUMPREG0_max
<dbl>,<dbl>,<dbl>,<dbl>
2.958093,1.978639,0,15


In [210]:
# Generate race dummy variables
data <- data %>% mutate(
  black = ifelse(RACE == 1, 1, 0),
  chinese = ifelse(RACE == 2, 1, 0),
  japanese = ifelse(RACE == 3, 1, 0),
  hispanic = ifelse(RACE == 5, 1, 0)
)

# Recode AGE0
data <- data %>% mutate(AGE0 = ifelse(is.na(AGE0), -9999, AGE0))

# Recode COMMITE0 and add labels
data <- data %>% mutate(COMMITE0 = case_when(
  COMMITE0 == -9 ~ -9999,
  COMMITE0 == 1 ~ 0,
  COMMITE0 == 2 ~ 1,
  TRUE ~ COMMITE0
))
commite_labels <- c("-9999" = "Missing", "0" = "No", "1" = "Yes")

data %>%
  filter(insample == 1) %>%
  count(AGE0)
data %>%
  filter(insample == 1) %>%
  count(COMMITE0)

AGE0,n
<dbl>,<int>
42,365
43,421
44,416
45,389
46,400
47,392
48,298
49,255
50,168


COMMITE0,n
<dbl+lbl>,<int>
0,754
1,2539


No missing values.

In [211]:
# Recode ENGAGSE0 and add labels
data <- data %>% mutate(ENGAGSE0 = case_when(
  ENGAGSE0 == -9 ~ -9999,
  ENGAGSE0 == 1 ~ 0,
  ENGAGSE0 == 2 ~ 1,
  is.na(ENGAGSE0) ~ -9999,
  TRUE ~ ENGAGSE0
))
engagse_labels <- c("-9999" = "Missing", "0" = "No", "1" = "Yes")

#ENGAGSE0 table
data %>%
  filter(insample == 1) %>%
  count(ENGAGSE0)


ENGAGSE0,n
<dbl+lbl>,<int>
-9999,154
0,672
1,2467


In [212]:
# Recode PRGNANC0 and add labels
data <- data %>% mutate(PRGNANC0 = case_when(
  PRGNANC0 %in% c(-9, -1) ~ -9999,
  PRGNANC0 == 1 ~ 0,
  PRGNANC0 == 2 ~ 1,
  is.na(PRGNANC0) ~ -9999,
  TRUE ~ PRGNANC0
))
prgnanc_labels <- c("-9999" = "Missing", "0" = "No", "1" = "Yes")

#PRGNANC0 table
data %>%
  filter(insample == 1) %>%
  count(PRGNANC0)

PRGNANC0,n
<dbl+lbl>,<int>
-9999,827
0,1762
1,704


In [213]:
# Update `insample` based on AGE and COMMITE0
data <- data %>% mutate(
  insample = ifelse(AGE0 == -9999 | COMMITE0 == -9999, 0, insample)
)

Due to the significant amount of missing values in INCOME0, ENGAGSE0, and PRGNANC0, dummy variables for the missing values will be made.

In [214]:
# Income dummy variables
data <- data %>% mutate(
  xincm = ifelse(INCOME0 == -9999, 1, 0),
  xinc1 = ifelse(INCOME0 == 1, 1, 0),
  xinc2 = ifelse(INCOME0 == 2, 1, 0),
  xinc3 = ifelse(INCOME0 == 3, 1, 0),
  xinc4 = ifelse(INCOME0 == 4, 1, 0)
)

# ENGAGSE0 dummy variables
data <- data %>% mutate(
  sexam = ifelse(ENGAGSE0 == -9999, 1, 0),
  sexa0 = ifelse(ENGAGSE0 == 0, 1, 0),
  sexa1 = ifelse(ENGAGSE0 == 1, 1, 0)
)

# PRGNANC0 dummy variables
data <- data %>% mutate(
  conm = ifelse(PRGNANC0 == -9999, 1, 0),
  con0 = ifelse(PRGNANC0 == 0, 1, 0),
  con1 = ifelse(PRGNANC0 == 1, 1, 0)
)

# Summary statistics
data_summary <- data %>% filter(insample == 1) %>% summarise(across(
  c(NUMPREG0, COMMITE0, AGE0, black, chinese, japanese, hispanic,
    xincm, xinc1, xinc2, xinc3, sexam, sexa0, sexa1, conm, con0),
  list(mean = mean, sd = sd, min = min, max = max), na.rm = TRUE
))
data_summary


NUMPREG0_mean,NUMPREG0_sd,NUMPREG0_min,NUMPREG0_max,COMMITE0_mean,COMMITE0_sd,COMMITE0_min,COMMITE0_max,AGE0_mean,AGE0_sd,⋯,sexa1_min,sexa1_max,conm_mean,conm_sd,conm_min,conm_max,con0_mean,con0_sd,con0_min,con0_max
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.958093,1.978639,0,15,0.7710295,0.4202341,0,1,45.84968,2.687478,⋯,0,1,0.2511388,0.433734,0,1,0.5350744,0.498844,0,1


## Analyzing the Data

The next step is to analyze our cleaned data to test our null hypothesis.

Our dependant variable, NUMPREG0, is a count variable of the number of pregnancies a woman has had. For this project, we start with a Poisson Regression Model and consider its suitability.
### Poisson

In [215]:
# Poisson Regression of insample
poisson_model <- glm(NUMPREG0 ~ AGE0 + COMMITE0 + xincm + xinc1 + xinc2 + xinc3 + black + hispanic +
    chinese + japanese + conm + con0 + sexam + sexa0, 
    family = poisson(), 
    data = data %>% filter(insample == 1))

# Checking model summary
summary(poisson_model)



Call:
glm(formula = NUMPREG0 ~ AGE0 + COMMITE0 + xincm + xinc1 + xinc2 + 
    xinc3 + black + hispanic + chinese + japanese + conm + con0 + 
    sexam + sexa0, family = poisson(), data = data %>% filter(insample == 
    1))

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.827388   0.180508   4.584 4.57e-06 ***
AGE0        -0.001251   0.003803  -0.329 0.742227    
COMMITE0     0.161838   0.032145   5.035 4.79e-07 ***
xincm        0.055230   0.067971   0.813 0.416470    
xinc1        0.238833   0.041521   5.752 8.82e-09 ***
xinc2        0.060701   0.034743   1.747 0.080617 .  
xinc3        0.012554   0.033432   0.376 0.707286    
black        0.291839   0.024727  11.803  < 2e-16 ***
hispanic     0.310124   0.038265   8.105 5.29e-16 ***
chinese      0.037451   0.041987   0.892 0.372418    
japanese     0.066197   0.040115   1.650 0.098904 .  
conm         0.069615   0.054702   1.273 0.203154    
con0         0.036421   0.026469   1.376 0.168832    
sexa

### Interpreting Poisson & Overdispersion

Before interpreting the results of the Poisson model, we must check for overdispersion to see if the data follows a Poisson distribution or Negative Binomial distribution. If overdispersion is present (variance>mean), then we will move on to the negative binomial regression model.

In [216]:
# Checking for overdispersion using Pearson residuals
residual_deviance <- sum(residuals(poisson_model, type = "pearson")^2)
overdispersion_ratio <- residual_deviance / poisson_model$df.residual
overdispersion_ratio


[1] 1.226987

The overdispersion ratio is slightly greater than 1, which indicates some overdispersion. A Negative Binomial Regression model would be a better fit.

### NBR

In [217]:
# Fit Negative Binomial Regression
negbinom_model <- glm.nb(NUMPREG0 ~ AGE0 + COMMITE0 + xincm + xinc1 + xinc2 + xinc3 + black + hispanic +
    chinese + japanese + conm + con0 + sexam + sexa0, 
                          data = data %>% filter(insample == 1))

# Check the summary of the Negative Binomial model
summary(negbinom_model)


Call:
glm.nb(formula = NUMPREG0 ~ AGE0 + COMMITE0 + xincm + xinc1 + 
    xinc2 + xinc3 + black + hispanic + chinese + japanese + conm + 
    con0 + sexam + sexa0, data = data %>% filter(insample == 
    1), init.theta = 15.75401484, link = log)

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.8060805  0.1972483   4.087 4.38e-05 ***
AGE0        -0.0008339  0.0041560  -0.201 0.840976    
COMMITE0     0.1646464  0.0350639   4.696 2.66e-06 ***
xincm        0.0551442  0.0742532   0.743 0.457693    
xinc1        0.2396043  0.0455110   5.265 1.40e-07 ***
xinc2        0.0611118  0.0377371   1.619 0.105359    
xinc3        0.0123074  0.0362470   0.340 0.734201    
black        0.2938626  0.0270417  10.867  < 2e-16 ***
hispanic     0.3113880  0.0423117   7.359 1.85e-13 ***
chinese      0.0377804  0.0454363   0.832 0.405690    
japanese     0.0668860  0.0434363   1.540 0.123593    
conm         0.0701003  0.0601504   1.165 0.243850    
con0         0.0357278  

In [218]:
# Overdispersion test
residual_deviance <- sum(residuals(refined_model, type = "pearson")^2)
overdispersion_ratio <- residual_deviance / refined_model$df.residual
overdispersion_ratio

[1] 1.037654

### Interpreting Negative Binomial Regression

The dispersion parameter, theta, is greater than 1, but the NBR model mostly accounts for this overdispersion, and when we repeat the overdispersion test the ratio is very close to 1. The model converged after one iteration. We see that this model is a better fit than Poisson, as the AIC is lower. We can now move on to interpreting results.

Significant variables are marked with asterisks. At a 95% confidence interval, we see that COMMITE0, xinc1, black, hispanic, and sexa0 are statistically significant. Our Null hypothesis was that there is no significant correlation between income, race, or other variables on the number of pregnancies. Our results contradict our Null hypothesis, and indicate that there *are* variables which significantly impact the number of pregnancies a woman is likely to have. In particular, being black, hispanic, or being in a commited relationship are strong positive predictors.

However, there are many variables that aren't statistically significant. The model can be refined by reducing the number of dummy variable categories by conflating them and/or dropping variables with high p-values, such as AGE0. (The non-significance of age to number of pregnancies is likely due to the narrow age-range of women sampled from.)


In [219]:
# Income dummy variables
data <- data %>% mutate(
  xinc1 = ifelse(INCOME0 == 1, 1, 0),
  xinc2.3.4 = ifelse(INCOME0 %in% c(2, 3, 4), 1, 0)
)

# ENGAGSE0 dummy variables
data <- data %>% mutate(
  sexa0 = ifelse(ENGAGSE0 == 0, 1, 0),
  sexa1 = ifelse(ENGAGSE0 == 1, 1, 0)
)

data <- data %>% mutate(
  black = ifelse(RACE == 1, 1, 0),
  asian = ifelse(RACE %in% c(2, 3), 1, 0),
  hispanic = ifelse(RACE == 5, 1, 0)
)

In [220]:
# Refined model without variables with high p-values (e.g., AGE0, xincm, sexam)
refined_model <- glm.nb(NUMPREG0 ~ COMMITE0 + xinc1 + xinc2.3.4 + black + hispanic +
    asian + sexa0, 
                          data = data %>% filter(insample == 1))
summary(refined_model)



Call:
glm.nb(formula = NUMPREG0 ~ COMMITE0 + xinc1 + xinc2.3.4 + black + 
    hispanic + asian + sexa0, data = data %>% filter(insample == 
    1), init.theta = 15.55707991, link = log)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.84984    0.07598  11.184  < 2e-16 ***
COMMITE0     0.15383    0.03444   4.466 7.97e-06 ***
xinc1        0.17910    0.07257   2.468   0.0136 *  
xinc2.3.4   -0.02044    0.06817  -0.300   0.7643    
black        0.30517    0.02656  11.490  < 2e-16 ***
hispanic     0.33318    0.04143   8.042 8.86e-16 ***
asian        0.04692    0.03352   1.400   0.1616    
sexa0       -0.18243    0.03600  -5.068 4.03e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Negative Binomial(15.5571) family taken to be 1)

    Null deviance: 4180.1  on 3292  degrees of freedom
Residual deviance: 3834.3  on 3285  degrees of freedom
AIC: 13104

Number of Fisher Scoring iterations: 1


              

The AIC has improved more, indicating a better fit. For the purpose of this project, this final model will be used to interpret the data.

In conclusion, low income, being black, being hispanic, being in a committed relationship, and *not* currently being sexually active (at the time the survey was taken) are all predictors of a high number of pregnancies.